In [4]:
import requests
from bs4 import BeautifulSoup as bs 
import re
import json

In [5]:
search_vacancy = input(f'Please enter vacancy that you are looking for: ')
url = 'https://russia.superjob.ru/vacancy/search/?keywords=' + search_vacancy
vacancy_list = []

with open('vacancies_data_tmp.json', 'w') as jf:
    while True:
        req = requests.get(url)
        if req.status_code == 200:
            soup = bs(req.content, 'html.parser')
            all_vacancies = soup.find_all('div', {'class': '_1M4pN _1O2dw _6kSO2 fmJDo'})
            for vacancy in all_vacancies:
                title = vacancy.find('span').find('a').text
                link = 'https://www.superjob.ru/' + vacancy.find('span').find('a')['href']
                salary = vacancy.find('span', {'class': '_4Gt5t _2jvwc f-test-text-company-item-salary'}).text
                if '—' in salary:
                    salary_min = salary.split('—')[0]
                    salary_min = re.sub(r'[^0-9]', '', salary_min)
                    salary_max = salary.split('—')[1]
                    salary_max = re.sub(r'[^0-9]', '', salary_max)
                    salary_min = int(salary_min)
                    salary_max = int(salary_max)
                elif 'от' in salary:
                    salary_min = salary[2:]
                    salary_min = re.sub(r'[^0-9]', '', salary_min)
                    salary_min = int(salary_min)
                    salary_max = None
                elif 'договорённости' in salary:
                    salary_min = None
                    salary_max = None
                elif 'до' in salary:
                    salary_min = None
                    salary_max = salary[2:]
                    salary_max = re.sub(r'[^0-9]', '', salary_max)
                    salary_max = int(salary_max)
                else:
                    salary_min = int(re.sub(r'[^0-9]', '', salary))
                    salary_max = int(re.sub(r'[^0-9]', '', salary))

                vacancy_dict = {
                    'title': title,
                    'salary_min': salary_min,
                    'salary_max': salary_max,
                    'link': link,
                    'website': 'https://www.superjob.ru/'

                }
                vacancy_list.append(vacancy_dict)
                json.dump(vacancy_dict, jf)
            try:
                next_page = soup.find('a', {'class': '_1IHWd _6Nb0L _37aW8 ljjt- f-test-button-dalshe f-test-link-Dalshe'})['href']
                url = 'https://russia.superjob.ru'+ next_page
                continue
            except:
                break
        else:
            print(f'Ошибка: {req.status_code}')
    print(f'\n{len(vacancy_list)} vacancies were found')
with open('vacancies_data.json', 'w') as jf:
    json.dump(vacancy_list, jf)

Please enter vacancy that you are looking for: Java

77 vacancies were found
